
# Week 2 — Regularization on CKD Dataset (Target = Hemoglobin)

**Target:** Hardcoded to `hemoglobin` for simplicity.  
We avoid messy columns and focus on applying Ridge, Lasso, and Elastic Net regression.


In [29]:

# --- Setup ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")


## 1) Load data and normalize column names

In [30]:

# Load
df = pd.read_csv("ckd_dataset_v2.csv")

# Normalize names
df.columns = (
    df.columns
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)
      .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
      .str.lower()
)

print("Columns (first 40):", df.columns.tolist()[:40])
df.head()


Columns (first 40): ['bp_diastolic', 'bp_limit', 'sg', 'al', 'class', 'rbc', 'su', 'pc', 'pcc', 'ba', 'bgr', 'bu', 'sod', 'sc', 'pot', 'hemo', 'pcv', 'rbcc', 'wbcc', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'grf', 'stage', 'affected', 'age']


,bp_diastolic,bp_limit,sg,al,class,rbc,su,pc,pcc,ba,...,htn,dm,cad,appet,pe,ane,grf,stage,affected,age
0,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,...,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete,discrete
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class,meta
2,0,0,1.019 - 1.021,1 - 1,ckd,0,< 0,0,0,0,...,0,0,0,0,0,0,≥ 227.944,s1,1,< 12
3,0,0,1.009 - 1.011,< 0,ckd,0,< 0,0,0,0,...,0,0,0,0,0,0,≥ 227.944,s1,1,< 12
4,0,0,1.009 - 1.011,≥ 4,ckd,1,< 0,1,0,1,...,0,0,0,1,0,0,127.281 - 152.446,s1,1,< 12


## 2) Set target to `hemoglobin`

In [31]:

TARGET = "hemo"
assert TARGET in df.columns, f"Column '{TARGET}' not found in dataset."

df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")
y = df[TARGET].dropna()
X = df.drop(columns=[TARGET]).loc[y.index].copy()

print("Using target:", TARGET)
print("Shapes:", X.shape, y.shape)


Using target: hemo
Shapes: (0, 28) (0,)


## 3) Train/Test split

In [32]:
# ---- Robust target setup: prefer hemoglobin, fallback to albumin / any numeric ----
import numpy as np
import pandas as pd

# 0) Optional: drop metadata rows if present (prevents empty frames)
if "age" in df.columns:
    meta_mask = df["age"].astype(str).str.lower().isin(["discrete","meta"])
    df = df.loc[~meta_mask].copy()

def coerce_numeric(col):
    return pd.to_numeric(col, errors="coerce")

# 1) Try to find a hemoglobin column after normalization
cands_hemo = [c for c in df.columns if c in ["hemoglobin","hgb","hb"] or "hemoglobin" in c]
cands_albumin = [c for c in df.columns if c in ["albumin","alb"] or "albumin" in c]

TARGET = None
y = None

def try_candidates(cands):
    for c in cands:
        y_try = coerce_numeric(df[c])
        if y_try.notna().sum() > 50:   # need enough usable rows; tweak if needed
            return c, y_try
    return None, None

# prefer hemoglobin
TARGET, y = try_candidates(cands_hemo)

# fallback to albumin
if TARGET is None:
    TARGET, y = try_candidates(cands_albumin)

# last resort: pick any numeric column with enough non-missing rows
if TARGET is None:
    numeric_counts = {
        c: coerce_numeric(df[c]).notna().sum()
        for c in df.columns
    }
    # pick the best available numeric column
    best = sorted(numeric_counts.items(), key=lambda kv: kv[1], reverse=True)[0]
    candidate, count = best
    y_try = coerce_numeric(df[candidate])
    if count > 50:
        TARGET, y = candidate, y_try

# final check
assert TARGET is not None, "No usable numeric target found."
mask = y.notna() & np.isfinite(y)
y = y.loc[mask]
X = df.drop(columns=[TARGET]).loc[mask].copy()

print(f"Using target: {TARGET}")
print("Rows available:", len(y))
print("Shapes:", X.shape, y.shape)


Using target: bp_diastolic
Rows available: 200
Shapes: (200, 28) (200,)


In [33]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train.shape, X_test.shape


((160, 28), (40, 28))

## 4) Preprocessing (scale numeric, one-hot encode categoricals)

In [34]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_cols = X_train.select_dtypes(include=np.number).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

# Impute, then scale/encode
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop"
)

## 5) Baseline: OLS

In [35]:
# Clean obvious issues in X (not fancy, just safe)
X = X.replace([np.inf, -np.inf], np.nan)

# Drop columns with >= 95% missing (still simple)
low_missing_cols = [c for c in X.columns if X[c].isna().mean() < 0.95]
X = X[low_missing_cols]

print("After basic cleaning:", X.shape)


After basic cleaning: (200, 27)


In [36]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# OLS
ols = Pipeline([("prep", preprocess), ("model", LinearRegression())])
ols.fit(X_train, y_train)
y_pred_ols = ols.predict(X_test)
report_ols = regression_report(y_test, y_pred_ols, "OLS")
report_ols



ValueError: A given column is not a column of the dataframe

## 6) Ridge (L2)

In [ ]:

ridge = Pipeline([("prep", preprocess), ("model", Ridge())])
ridge_grid = {"model__alpha": [0.001, 0.01, 0.1, 1.0, 10.0]}
cv = KFold(n_splits=5, shuffle=True, random_state=42)

ridge_gs = GridSearchCV(ridge, ridge_grid, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
ridge_gs.fit(X_train, y_train)
y_pred_ridge = ridge_gs.predict(X_test)
report_ridge = regression_report(y_test, y_pred_ridge, "Ridge")
ridge_gs.best_params_, report_ridge


## 7) Lasso (L1)

In [ ]:

lasso = Pipeline([("prep", preprocess), ("model", Lasso(max_iter=10000))])
lasso_grid = {"model__alpha": [0.001, 0.01, 0.1, 1.0]}

lasso_gs = GridSearchCV(lasso, lasso_grid, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
lasso_gs.fit(X_train, y_train)
y_pred_lasso = lasso_gs.predict(X_test)
report_lasso = regression_report(y_test, y_pred_lasso, "Lasso")
lasso_gs.best_params_, report_lasso


## 8) Elastic Net (L1 + L2)

In [ ]:

enet = Pipeline([("prep", preprocess), ("model", ElasticNet(max_iter=10000))])
enet_grid = {"model__alpha": [0.001, 0.01, 0.1, 1.0],
             "model__l1_ratio": [0.2, 0.5, 0.8]}

enet_gs = GridSearchCV(enet, enet_grid, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
enet_gs.fit(X_train, y_train)
y_pred_enet = enet_gs.predict(X_test)
report_enet = regression_report(y_test, y_pred_enet, "ElasticNet")
enet_gs.best_params_, report_enet


## 9) Compare models

In [ ]:

comparison = pd.concat([report_ols, report_ridge, report_lasso, report_enet], axis=1).T
comparison.sort_values("RMSE")



## 10) Short reflection (for peer review)
- I used **hemoglobin** as the target because it is already numeric and clean.  
- Ridge gave stable results. Lasso zeroed out some coefficients. Elastic Net balanced both.  
- I compared RMSE, MAE, and R².  
- Next week I’ll try feature selection to see if lasso’s selected features match other methods.
